In [1]:
import json, hashlib, itertools
from falkordb import FalkorDB

In [5]:
def sid(s):  # stable id from title
    return hashlib.sha1(s.encode()).hexdigest()[:12]

# json extraction
def extract_first_json(text):
    start = text.find('{')
    if start == -1:
        return None
    count = 0
    for i in range(start, len(text)):
        if text[i] == '{':
            count += 1
        elif text[i] == '}':
            count -= 1
            if count == 0:
                return json.loads(text[start:i+1])
    return None

In [14]:
file_path = 'json/2307.16513v2_strict_2p_o3.json'
file_path = 'traces/2307.16513v2_flex_1p_o3.txt'

with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            data = extract_first_json(content)

In [6]:
data

{'nodes': [{'name': 'emergent deception abilities in state-of-the-art LLMs',
   'aliases': ['LLM deceptive behaviour', 'false-belief inducing capacity'],
   'type': 'concept',
   'description': 'Recent large language models (ChatGPT-3.5, GPT-4) can intentionally induce false beliefs in tests.',
   'concept_category': 'Finding',
   'intervention_lifecycle': None,
   'intervention_maturity': None},
  {'name': 'potential for models to bypass human monitoring and alignment',
   'aliases': ['alignment bypass risk', 'monitoring evasion threat'],
   'type': 'concept',
   'description': 'If models can deceive evaluators, they may hide unsafe objectives or behaviors.',
   'concept_category': 'Risk',
   'intervention_lifecycle': None,
   'intervention_maturity': None},
  {'name': 'implement standardized deception-evaluation suite during pre-deployment testing',
   'aliases': ['deception benchmark before release',
    'pre-deployment deception audit'],
   'type': 'intervention',
   'description':

In [22]:
file_path = '../intervention_graph_creation/prompt/schemas/output_sample.json'

with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            data = json.loads(content)

print(data['nodes'][0]['type'].strip())

# --- build nodes ---
nodes = []
for n in data["nodes"]:
    nodes.append({
        "label": n["type"].strip(),
        "props": {
            "name": n["name"].strip(),
            "aliases": n.get("aliases", []),
            "description": n.get("description", ""),
            "concept_category": n.get("concept_category", ""),
            "maturity": n.get("maturity"),
            "intervention_lifecycle": n.get("intervention_lifecycle"),
            "intervention_maturity": n.get("intervention_maturity"),
        }
    })

print(nodes)

concept
[{'label': 'concept', 'props': {'name': 'emergent deception abilities in state-of-the-art LLMs', 'aliases': ['LLM deceptive behaviour', 'false-belief inducing capacity'], 'description': 'Recent large language models (ChatGPT-3.5, GPT-4) can intentionally induce false beliefs in tests.', 'concept_category': 'Finding', 'maturity': None, 'intervention_lifecycle': None, 'intervention_maturity': None}}, {'label': 'concept', 'props': {'name': 'potential for models to bypass human monitoring and alignment', 'aliases': ['alignment bypass risk', 'monitoring evasion threat'], 'description': 'If models can deceive evaluators, they may hide unsafe objectives or behaviors.', 'concept_category': 'Risk', 'maturity': None, 'intervention_lifecycle': None, 'intervention_maturity': None}}, {'label': 'intervention', 'props': {'name': 'implement standardized deception-evaluation suite during pre-deployment testing', 'aliases': ['deception benchmark before release', 'pre-deployment deception audit']

In [23]:
# --- build edges ---
edges = []
for chain in data.get("logical_chains", []):
    for e in chain.get("edges", []):
        retype = e["type"].upper().replace("-", "_")
        edges.append({
            "type": retype,
            "row": {
                "src": e["source_node"].strip(),
                "dst": e["target_node"].strip(),
                "description": e.get("description", ""),
                "confidence": e.get("confidence")
            }
        })

print(edges)

[{'type': 'LEADS_TO', 'row': {'src': 'emergent deception abilities in state-of-the-art LLMs', 'dst': 'potential for models to bypass human monitoring and alignment', 'description': 'If models can deceive, they might hide unsafe intentions.', 'confidence': None}}, {'type': 'MITIGATED_BY', 'row': {'src': 'potential for models to bypass human monitoring and alignment', 'dst': 'implement standardized deception-evaluation suite during pre-deployment testing', 'description': 'Systematic deception testing reduces unnoticed misalignment.', 'confidence': None}}, {'type': 'LEADS_TO', 'row': {'src': 'chain-of-thought prompting amplifies deception performance', 'dst': 'potential for models to bypass human monitoring and alignment', 'description': 'Amplified deception exacerbates alignment risk.', 'confidence': None}}, {'type': 'MITIGATED_BY', 'row': {'src': 'chain-of-thought prompting amplifies deception performance', 'dst': 'restrict or obfuscate chain-of-thought outputs in deployed systems', 'de

In [ ]:
# --- 3) insert into FalkorDB ---
db = FalkorDB(host="localhost", port=6379)
g = db.select_graph("misalignment_v3")

# indexes (safe to re-run)
g.query("CREATE INDEX FOR (c:Concept) ON (c.title)")
g.query("CREATE INDEX FOR (i:Intervention) ON (i.title)")

# nodes (group by label)
by_label = defaultdict(list)
for n in nodes:
    by_label[n["label"]].append(n["props"])

In [ ]:

# for L, rows in by_label.items():
#     for chunk in batched(rows):
#         g.query(f"""
#         UNWIND $rows AS row
#         MERGE (n:`{L}` {{title: row.title}})
#         SET n += row
#         """, {"rows": chunk})

# # relationships (group by type)
# by_type = defaultdict(list)
# for e in edges:
#     # keep only rel props in the payload; src/dst are separate keys
#     row = e["row"]
#     by_type[e["type"]].append(row)

# for T, rows in by_type.items():
#     for chunk in batched(rows):
#         g.query(f"""
#         UNWIND $rows AS row
#         MATCH (s {{title: row.src}}), (t {{title: row.dst}})
#         MERGE (s)-[r:`{T}`]->(t)
#         SET r.description = row.description,
#             r.confidence = row.confidence
#         """, {"rows": chunk})

# print("Done. Nodes keyed by title; 'name' mirrors title for UI display.")

AttributeError: 'NoneType' object has no attribute 'strip'

In [23]:
# graph from flex json
file_path = 'traces/2307.16513v2_flex_1p_o3.txt'

with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            data = extract_first_json(content)

nodes = []
name2id = {}
labels_seen = set()

for n in data["nodes"]:
    key = (n.get("canonical_name") or n["name"]).strip()
    nid = sid(key)
    label = (n["type"] or "ENTITY").upper()   # e.g., TASK, MODEL, PROMPT_TECHNIQUE
    labels_seen.add(label)
    name2id[n["name"]] = nid
    if n.get("canonical_name"): name2id[n["canonical_name"]] = nid  # allow both
    nodes.append({
        "label": label,
        "props": {
            "id": nid,
            "name": n["name"],
            "canonical_name": n.get("canonical_name"),
            "aliases": n.get("aliases", []),
            "confidence": n.get("confidence"),
            "notes": n.get("notes", "")
        }
    })

# --- 2) Relationships from logical_chains ---
rels = []
for chain in data.get("logical_chains", []):
    for e in chain.get("edges", []):
        reltype = (e["type"] or "RELATED").upper().replace("-", "_")
        src = name2id[e["source_node"]]
        dst = name2id[e["target_node"]]
        rels.append({
            "type": reltype,
            "row": {
                "src": src, "dst": dst,
                "rationale": e.get("rationale", ""),
                "confidence": e.get("confidence")
            }
        })

# --- 3) Insert into FalkorDB ---
db = FalkorDB(host="localhost", port=6379)
g = db.select_graph("misalignment_v2")

# indexes (id lookups fast; safe to run repeatedly)
for L in labels_seen:
    g.query(f"CREATE INDEX FOR (n:`{L}`) ON (n.id)")

# nodes (group by label)
by_label = defaultdict(list)
for n in nodes:
    by_label[n["label"]].append(n["props"])

for L, rows in by_label.items():
    for chunk in batched(rows):
        g.query(f"""
        UNWIND $rows AS row
        MERGE (n:`{L}` {{id: row.id}})
        SET n += row
        """, {"rows": chunk})

# relationships (group by type)
by_type = defaultdict(list)
for r in rels:
    by_type[r["type"]].append(r["row"])

for T, rows in by_type.items():
    for chunk in batched(rows):
        g.query(f"""
        UNWIND $rows AS row
        MATCH (s {{id: row.src}}), (t {{id: row.dst}})
        MERGE (s)-[r:`{T}`]->(t)
        SET r += row
        """, {"rows": chunk})

print("Done.")

Done.
